# **Finding Lane Lines on the Road** 
***
In this project, you will use the tools you learned about in the lesson to identify lane lines on the road.  You can develop your pipeline on a series of individual images, and later apply the result to a video stream (really just a series of images). Check out the video clip "raw-lines-example.mp4" (also contained in this repository) to see what the output should look like after using the helper functions below. 

Once you have a result that looks roughly like "raw-lines-example.mp4", you'll need to get creative and try to average and/or extrapolate the line segments you've detected to map out the full extent of the lane lines.  You can see an example of the result you're going for in the video "P1_example.mp4".  Ultimately, you would like to draw just one line for the left side of the lane, and one for the right.

---
Let's have a look at our first image called 'test_images/solidWhiteRight.jpg'.  Run the 2 cells below (hit Shift-Enter or the "play" button above) to display the image.

**Note** If, at any point, you encounter frozen display windows or other confounding issues, you can always start again with a clean slate by going to the "Kernel" menu above and selecting "Restart & Clear Output".

---

**The tools you have are color selection, region of interest selection, grayscaling, Gaussian smoothing, Canny Edge Detection and Hough Tranform line detection.  You  are also free to explore and try other techniques that were not presented in the lesson.  Your goal is piece together a pipeline to detect the line segments in the image, then average/extrapolate them and draw them onto the image for display (as below).  Once you have a working pipeline, try it out on the video stream below.**

---

<figure>
 <img src="line-segments-example.jpg" width="380" alt="Combined Image" />
 <figcaption>
 <p></p> 
 <p style="text-align: center;"> Your output should look something like this (above) after detecting line segments using the helper functions below </p> 
 </figcaption>
</figure>
 <p></p> 
<figure>
 <img src="laneLines_thirdPass.jpg" width="380" alt="Combined Image" />
 <figcaption>
 <p></p> 
 <p style="text-align: center;"> Your goal is to connect/average/extrapolate line segments to get output like this</p> 
 </figcaption>
</figure>

In [1]:
#importing some useful packages
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import cv2
%matplotlib inline

**Some OpenCV functions (beyond those introduced in the lesson) that might be useful for this project are:**

`cv2.inRange()` for color selection  
`cv2.fillPoly()` for regions selection  
`cv2.line()` to draw lines on an image given endpoints  
`cv2.addWeighted()` to coadd / overlay two images
`cv2.cvtColor()` to grayscale or change color
`cv2.imwrite()` to output images to file  
`cv2.bitwise_and()` to apply a mask to an image

**Check out the OpenCV documentation to learn about these and discover even more awesome functionality!**

Below are some helper functions to help get you started. They should look familiar from the lesson!

In [2]:
import math

def grayscale(img):
    """Applies the Grayscale transform
    This will return an image with only one color channel
    but NOTE: to see the returned image as grayscale
    you should call plt.imshow(gray, cmap='gray')"""
    return cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    # Or use BGR2GRAY if you read an image with cv2.imread()
    # return cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
def canny(img, low_threshold, high_threshold):
    """Applies the Canny transform"""
    return cv2.Canny(img, low_threshold, high_threshold)

def gaussian_blur(img, kernel_size):
    """Applies a Gaussian Noise kernel"""
    return cv2.GaussianBlur(img, (kernel_size, kernel_size), 0)

def region_of_interest(img, vertices):
    """
    Applies an image mask.
    
    Only keeps the region of the image defined by the polygon
    formed from `vertices`. The rest of the image is set to black.
    """
    #defining a blank mask to start with
    mask = np.zeros_like(img)   
    
    #defining a 3 channel or 1 channel color to fill the mask with depending on the input image
    if len(img.shape) > 2:
        channel_count = img.shape[2]  # i.e. 3 or 4 depending on your image
        ignore_mask_color = (255,) * channel_count
    else:
        ignore_mask_color = 255
        
    #filling pixels inside the polygon defined by "vertices" with the fill color    
    cv2.fillPoly(mask, vertices, ignore_mask_color)
    
    #returning the image only where mask pixels are nonzero
    masked_image = cv2.bitwise_and(img, mask)
    return masked_image

def max_point(lines):
    max = 0
    max_point = []
    for line in lines:
        for x1,y1,x2,y2 in line:
            if (x1+y1) > max:
                max = x1+y1
                max_point = [x1,y1]
            if (x2+y2) > max:
                max = x2+y2
                max_point = [x2,y2]
    return max_point

def min_point(lines):
    min = sum(max_point(lines))
    min_point = []
    for line in lines:
        for x1,y1,x2,y2 in line:
            if (x1+y1) < min:
                min = x1+y1
                min_point = [x1,y1]
            if (x2+y2) < min:
                min = x2+y2
                min_point = [x2,y2]
    return min_point

def slope(line):
    slope = float(line[0][3] - line[0][1]) / float(line[0][2] - line[0][0])
    return slope

def slope_lines(lines):
    #remove any lines with zero slopes; causes division by zero exception later
    non_zero_slope_lines = lines
    i = 0
    for line in lines:
        if (slope(line) == 0):
            non_zero_slope_lines = np.delete(non_zero_slope_lines, i, axis=0)
        else:
            i += 1
   
    #remove any lines with almost zero slopes - lines close to horizontal
    non_horizontal_slope_lines = non_zero_slope_lines
    i = 0
    for line in non_zero_slope_lines:
        if (abs(slope(line)) < 0.35):
            non_horizontal_slope_lines = np.delete(non_horizontal_slope_lines, i, axis=0)
        else:
            i += 1
        
    pos_slope_lines = non_horizontal_slope_lines
    neg_slope_lines = non_horizontal_slope_lines
    pos = 0
    neg = 0
    for line in non_horizontal_slope_lines:
        if (slope(line) > 0):
            neg_slope_lines = np.delete(neg_slope_lines, pos, axis=0)
            neg += 1
        else:
            pos_slope_lines = np.delete(pos_slope_lines, neg, axis=0) 
            pos += 1
    return non_horizontal_slope_lines, pos_slope_lines, neg_slope_lines
 
def draw_lines(img, lines, color=[255, 0, 0], thickness=10): 
    global FL, NL, PL
    valid_lines, pos_lines, neg_lines = slope_lines(lines)
    if (np.sum(FL) == 0):
        FL = valid_lines
    if (np.sum(NL) == 0):
        NL = neg_lines
    if (np.sum(PL) == 0):
        PL = pos_lines
    if ((len(pos_lines) == 0) and (len(neg_lines) == 0)):
        if(len(FL) == 0):
            return
        for line in FL:
            for x1,y1,x2,y2 in line:
                cv2.line(img, (x1, y1), (x2, y2), color, thickness)
        return
    if ((len(pos_lines) == 0) or (len(neg_lines) == 0)):
        if ((len(pos_lines) != 0)):
            max_pt = max_point(pos_lines)
            min_pt = min_point(pos_lines)
            final_line1 = np.concatenate((min_pt, max_pt))
            final_line2 = NL
            final_lines = np.concatenate(([[final_line1]], final_line2))
            FL, PL, NL = slope_lines(final_lines)
            for line in final_lines:
                for x1,y1,x2,y2 in line:
                    cv2.line(img, (x1, y1), (x2, y2), color, thickness)
            return
        else:
            max_pt = max_point(neg_lines)
            min_pt = min_point(neg_lines)
            final_line1 = np.concatenate((min_pt, max_pt))
            final_line2 = PL
            final_lines = np.concatenate(([[final_line1]], final_line2))
            FL, PL, NL = slope_lines(final_lines)
            for line in final_lines:
                for x1,y1,x2,y2 in line:
                    cv2.line(img, (x1, y1), (x2, y2), color, thickness)
            return

    if (len(pos_lines) > len(neg_lines)):
        max_pt = max_point(pos_lines)
        min_pt = min_point(pos_lines)
        final_line1 = np.concatenate((min_pt, max_pt))
        neg_pt = [neg_lines[0][0][0], neg_lines[0][0][1]]
        x_max = int(float(max_pt[1] - neg_pt[1])/slope(neg_lines[0]) + neg_pt[0])
        y_max = max_pt[1]
        x_min = int(float(min_pt[1] - neg_pt[1])/slope(neg_lines[0]) + neg_pt[0])
        y_min = min_pt[1]
        final_line2 = np.array([[[x_min, y_min, x_max, y_max]]])
        final_lines = np.concatenate(([[final_line1]], final_line2))
        FL, PL, NL = slope_lines(final_lines)
    else:
        max_pt = max_point(neg_lines)
        min_pt = min_point(neg_lines)
        final_line1 = np.concatenate((min_pt, max_pt))
        pos_pt = [pos_lines[0][0][0], pos_lines[0][0][1]]
        x_max = int(float(max_pt[1] - pos_pt[1])/slope(pos_lines[0]) + pos_pt[0])
        y_max = max_pt[1]
        x_min = int(float(min_pt[1] - pos_pt[1])/slope(pos_lines[0]) + pos_pt[0])
        y_min = min_pt[1]
        final_line2 = np.array([[[x_min, y_min, x_max, y_max]]])
        final_lines = np.concatenate(([[final_line1]], final_line2))
        FL, PL, NL = slope_lines(final_lines)

    for line in final_lines:
        for x1,y1,x2,y2 in line:
            cv2.line(img, (x1, y1), (x2, y2), color, thickness)
        
def hough_lines(img, rho, theta, threshold, min_line_len, max_line_gap):
    """
    `img` should be the output of a Canny transform.
        
    Returns an image with hough lines drawn.
    """
    line_img = np.zeros((*img.shape, 3), dtype=np.uint8)
    lines = cv2.HoughLinesP(img, rho, theta, threshold, np.array([]), minLineLength=min_line_len, maxLineGap=max_line_gap)
    if lines is None:
         return line_img
    draw_lines(line_img, lines)
    return line_img

# Python 3 has support for cool math symbols.

def weighted_img(img, initial_img, α=0.8, β=1., λ=0.):
    """
    `img` is the output of the hough_lines(), An image with lines drawn on it.
    Should be a blank image (all black) with lines drawn on it.
    
    `initial_img` should be the image before any processing.
    
    The result image is computed as follows:
    
    initial_img * α + img * β + λ
    NOTE: initial_img and img must be the same shape!
    """
    return cv2.addWeighted(initial_img, α, img, β, λ)

In [3]:
def process_image(image):
    # NOTE: The output you return should be a color image (3 channel) for processing video below
    # TODO: put your pipeline here,
    # you should return the final output (image with lines are drawn on lanes)
    
    # Read in the image
    #initial_image = mpimg.imread(image)
    if (type(image).__module__ == np.__name__):
        initial_image = image
    else:
        initial_image = mpimg.imread(image)
        
    img_shape = initial_image.shape

    # Convert the image to grayscale
    gray = grayscale(initial_image)

    # Define a kernel size and apply Gaussian smoothing
    kernel_size = 5
    blur_gray = gaussian_blur(gray, kernel_size)

    # Define our parameters for Canny and apply
    low_threshold = 50
    high_threshold = 150
    edges = canny(blur_gray, low_threshold, high_threshold)

    # Defining a four sided polygon to mask
    #vertices = np.array([[(100, 540), (480, 300), (480,150), (960-50,540)]], dtype=np.int32)
    vertices = np.array([[(100, img_shape[0]),(img_shape[1]/2, (img_shape[0]/2)+30), (img_shape[1]/2, (img_shape[0]/2)+30), (img_shape[1]-50,img_shape[0])]], dtype=np.int32)
    masked_edges = region_of_interest(edges, vertices)

    # Define the Hough transform parameters, run Hough on edge detected image
    # Return lines detected by Hough
    rho = 1 # distance resolution in pixels of the Hough grid
    theta = np.pi/360 # angular resolution in radians of the Hough grid, 60, 270, 180, 360
    threshold = 35     # minimum number of votes (intersections in Hough grid cell)
    min_line_length = 40 #minimum number of pixels making up a line
    max_line_gap = 1    # maximum gap in pixels between connectable line segments
    line_image = hough_lines(masked_edges, rho, theta, threshold, min_line_length, max_line_gap)

    # Overlay the detected line on the initial image
    final_image = weighted_img(line_image, initial_image, α=0.8, β=1., λ=0.)
    
    return final_image

## Test on Images

Now you should build your pipeline to work on the images in the directory "test_images"  
**You should make sure your pipeline works well on these images before you try the videos.**

In [4]:
import os
image_file_list = os.listdir("test_images/")
print("image files", image_file_list)

image files ['solidWhiteCurve.jpg', 'solidWhiteRight.jpg', 'solidYellowCurve.jpg', 'solidYellowCurve2.jpg', 'solidYellowLeft.jpg', 'whiteCarLaneSwitch.jpg']


run your solution on all test_images and make copies into the test_images directory).

In [5]:
#Global initialization. NL is Negative slope lines, PL is positive slope lines, FL is final lines. This is used to interpolate
NL = np.array([[[0,0,0,0]]])
PL = np.array([[[0,0,0,0]]])
FL = np.concatenate((NL,PL))

In [6]:
for image_file in image_file_list:
    image_file_and_dir = "test_images/"+image_file
    output_file = os.path.splitext(image_file_and_dir)[0]+"_output.jpg"
    fnl = process_image(image_file_and_dir)
    mpimg.imsave(output_file, fnl)

## Test on Videos

You know what's cooler than drawing lanes over images? Drawing lanes over video!

We can test our solution on two provided videos:

`solidWhiteRight.mp4`

`solidYellowLeft.mp4`

Let's try the one with the solid white lane on the right first ...

In [7]:
# Import everything needed to edit/save/watch video clips
from moviepy.editor import VideoFileClip
from IPython.display import HTML

In [8]:
white_output = 'white.mp4'
clip1 = VideoFileClip("solidWhiteRight.mp4")
white_clip = clip1.fl_image(process_image) #NOTE: this function expects color images!!
%time white_clip.write_videofile(white_output, audio=False)

[MoviePy] >>>> Building video white.mp4
[MoviePy] Writing video white.mp4


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▎| 221/222 [00:09<00:00, 24.14it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: white.mp4 

Wall time: 10.8 s


Play the video inline, or if you prefer find the video in your filesystem (should be in the same directory) and play it in your video player of choice.

In [9]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(white_output))

**At this point, if you were successful you probably have the Hough line segments drawn onto the road, but what about identifying the full extent of the lane and marking it clearly as in the example video (P1_example.mp4)?  Think about defining a line to run the full length of the visible lane based on the line segments you identified with the Hough Transform.  Modify your draw_lines function accordingly and try re-running your pipeline.**

Now for the one with the solid yellow lane on the left. This one's more tricky!

In [10]:
yellow_output = 'yellow.mp4'
clip2 = VideoFileClip('solidYellowLeft.mp4')
yellow_clip = clip2.fl_image(process_image)
%time yellow_clip.write_videofile(yellow_output, audio=False)

[MoviePy] >>>> Building video yellow.mp4
[MoviePy] Writing video yellow.mp4


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▊| 681/682 [00:30<00:00, 22.22it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: yellow.mp4 

Wall time: 32.3 s


In [11]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(yellow_output))

## Reflections

Congratulations on finding the lane lines!  As the final step in this project, we would like you to share your thoughts on your lane finding pipeline... specifically, how could you imagine making your algorithm better / more robust?  Where will your current algorithm be likely to fail?

Please add your thoughts below,  and if you're up for making your pipeline more robust, be sure to scroll down and check out the optional challenge video below!


## Kris Harikrishnan Reflections

I've completed all the assignments including the challenge question below.

Some comments on my work:
* If I could not detect any lanes in the current frame, I extrapolated the lane (lines) from the previos frame
* If one of the lanes (left or right) could not be detected, I extrapolated the missing lane (line) from the previous frame
* If one of the lines (left or right) was shorter than the other I extrapolated to make them about the same length
* I removed all the lines (such as horizontal) other than the lane lines

Some reflections:
* Hard to detect lanes in the shady part of the road
* Not sure how this code will work on a road with no lane markings but just the gutter
* Not sure how this code will work in a center lane of a multi-lane freeway assuming there are no solid lane lines on both sides

## Submission

If you're satisfied with your video outputs it's time to submit!  Submit this ipython notebook for review.


## Optional Challenge

Try your lane finding pipeline on the video below.  Does it still work?  Can you figure out a way to make it more robust?  If you're up for the challenge, modify your pipeline so it works with this video and submit it along with the rest of your project!

In [12]:
challenge_output = 'extra.mp4'
clip2 = VideoFileClip('challenge.mp4')
challenge_clip = clip2.fl_image(process_image)
%time challenge_clip.write_videofile(challenge_output, audio=False)

[MoviePy] >>>> Building video extra.mp4
[MoviePy] Writing video extra.mp4


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 251/251 [00:24<00:00, 11.83it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: extra.mp4 

Wall time: 27.1 s


In [13]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(challenge_output))